In [1]:
# Assignment 3 Data Engineering 5hp
# By Ture Hassler
# Initialize spark session and spark context

from pyspark.sql import SparkSession


ss = SparkSession \
    .builder \
    .master("spark://192.168.2.122:7077") \
    .appName("turehassler_partA") \
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
    .config("spark.shuffle.service.enabled", True)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .config("spark.cores.max", 16)\
    .getOrCreate()

sc = ss.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/23 15:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# A1, Read data and print nr lines and nr partitions
rdd_en = sc.textFile("hdfs://host-192-168-2-122:9000/europarl/europarl-v7.sv-en.en")
rdd_sv = sc.textFile("hdfs://host-192-168-2-122:9000/europarl/europarl-v7.sv-en.sv")

nrLinesEng = rdd_en.count()
nrPartEng = rdd_en.getNumPartitions()
nrLinesSwe = rdd_sv.count()
nrPartSwe = rdd_sv.getNumPartitions()

print("Nr lines. Eng: ", nrLinesEng, " Swe: ", nrLinesSwe)
print("Nr partitions. Eng: ", nrPartEng, "Swe: ", nrPartSwe)



Nr lines. Eng:  1862234  Swe:  1862234
Nr partitions. Eng:  2 Swe:  3


In [3]:
# A2, preprocess data, lower and split 
rdd_en_2 = rdd_en.map(lambda x: x.lower().split()) 
rdd_sv_2 = rdd_sv.map(lambda x: x.lower().split())

out1 = rdd_en_2.take(10)
out2 = rdd_sv_2.take(10)
lc1 = rdd_en_2.count()
lc2 = rdd_sv_2.count()

print("First 10 sentences:")
print(out1)
print(out2)
print("Nr lines: ")
print(lc1)
print(lc2)

First 10 sentences:
[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on

In [4]:
# A3, count most frequent words

# turn into keyvalue pairs and reduce (sum them up)
from operator import add

rdd_en_3 = rdd_en_2.flatMap(lambda x: x).map(lambda x: (x,1))
rdd_sv_3 = rdd_sv_2.flatMap(lambda x: x).map(lambda x: (x,1))

rdd_en_4 = rdd_en_3.reduceByKey(add)
rdd_en_5 = rdd_en_4.sortBy(lambda x: x[1], False).take(10)

rdd_sv_4 = rdd_sv_3.reduceByKey(add).sortBy(lambda x: x[1], False).take(10) # False added for reverse order

print("Most frequently ocuring words: ")
print("Eng:")
for kv in rdd_en_5:
    print(kv)
print("Swe:")
for kv in rdd_sv_4:
    print(kv)

# Seems reasonble i guess? Makes sense that "the" is the most frequent word. 

Most frequently ocuring words: 
Eng:
('the', 3498574)
('of', 1659884)
('to', 1539823)
('and', 1288620)
('in', 1086089)
('that', 797576)
('a', 773812)
('is', 758087)
('for', 534270)
('we', 522879)
Swe:
('att', 1706309)
('och', 1344895)
('i', 1050989)
('det', 924878)
('som', 913302)
('för', 908703)
('av', 738102)
('är', 694389)
('en', 620347)
('vi', 539808)


In [8]:
# A4
# Zip sentences with row index
a4_en1 = rdd_en_2.zipWithIndex().map(lambda x: (x[1], x[0]))
a4_sv1 = rdd_sv_2.zipWithIndex().map(lambda x: (x[1], x[0]))

# Inner join on sentences, then several filters 
combined = a4_en1.join(a4_sv1).map(lambda x: x[1])
c2 = combined.filter(lambda x: x[0] is not None and x[1] is not None)
c3 = c2.filter(lambda x: len(x[0]) == len(x[1]))
c4 = c3.filter(lambda x: len(x[0]) <= 5)
c5 = c4.flatMap(lambda x: zip(x[0], x[1])) # Pair the words in each sentence
c6 = c5.map(lambda x: (x, 1)).reduceByKey(add).sortBy(lambda x: x[1], False) # Add index and count occurences

print(c6.take(10))

# Translations are more accurate than i thought! Looks good.


[(('is', 'är'), 4718), (('closed.', 'avslutad.'), 2971), (('(applause)', '(applåder)'), 2546), (('.', '.'), 2107), (('that', 'det'), 1495), (('we', 'vi'), 1443), (('the', 'jag'), 1336), (('is', 'debatten'), 1327), (('debate', 'förklarar'), 1319), (('the', 'debatten'), 1253)]


In [ ]:
spark_session.stop()